In [2]:
%reload_ext line_profiler
%reload_ext Cython

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend("TkAgg")
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

from interpolated_functions import interpolated_mod2pi
sin_interpolated = interpolated_mod2pi(np.sin, -2.1 * np.pi, 2.1 * np.pi, 2000000)    
cos_interpolated = interpolated_mod2pi(np.cos, -2.1 * np.pi, 2.1 * np.pi, 2000000)

import seaborn as sns
sns.set_context('notebook', font_scale=1.3,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Python sin
---

In [4]:
def pysin(x):
    y = [math.sin(i) for i in x]
    return y

## Numpy sin
---

In [5]:
import numpy as np

def npsin(x):
    return np.sin(x)

## VDT sin - sinv - sincos
---

In [6]:
%%cython -I./vdt/include -L./vdt/lib --lib=vdt --compile-args=-fopenmp --link-args=-fopenmp --cplus
#cython boundscheck=False

cimport cython
import numpy as np
cimport numpy as np
from cython.parallel import prange

cdef extern from "sin.h" namespace "vdt":
    cdef double fast_sin(double x) nogil
    cdef void fast_sinv(int n, double *x, double *s) nogil

cdef extern from "sincos.h" namespace "vdt":
    cdef void fast_sincos(double x, double &s, double &c) nogil

@cython.boundscheck(False)
def vdt_sin(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=4):
        s[i] = fast_sin(x[i])
    
@cython.boundscheck(False)
def vdt_sincos(double[::1] x, double[::1] s, double[::1] c):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=4):
        fast_sincos(x[i], s[i], c[i])

@cython.boundscheck(False)
def vdt_sinv(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    fast_sinv(n, &x[0], &s[0])

## math.h sincos
---

In [7]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import prange

cdef extern from "math.h":
    cdef void sincos(double x, double *sin, double *cos) nogil
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(False)
cpdef fastsincos(double[::1] x, double[::1] s, double[::1] c):
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=4):
        sincos(x[i], &s[i], &c[i])

## cmath sin
---

In [8]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import prange
from libc.math cimport sin

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(False)
def cmsin(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in prange(n, nogil=True, num_threads=4):
        s[i] = sin(x[i])

In [9]:
n = int(1e6)
x = np.linspace(0, 2*np.pi, n)
xd = np.zeros(len(x)*2)
a = np.zeros(n)
b = np.zeros(n)

# d = np.zeros(2*n)
# xd[::2] = x[:]

In [11]:
pysin_r = pysin(x)
npsin_r = npsin(x)
cmsin(x, a); cmsin_r = a.copy()

fastsincos(x, a, b); sincos_r = a.copy()
interpsin_r = sin_interpolated(x)

vdt_sin(x, a); vdt_sin_r = a.copy()
vdt_sinv(x, a); vdt_sinv_r = a.copy()
vdt_sincos(x, a, b); vdt_sincos_r = a.copy()

fig, axes = plt.subplots(2, 3, figsize=(16,9), sharex=True, tight_layout=True)

axes[0,0].plot(x, npsin_r - cmsin_r)
axes[0,0].set_title("Numpy sin vs. cmath sin")
axes[0,1].plot(x, npsin_r-sincos_r)
axes[0,1].set_title("Numpy sin vs. cmath sincos")
axes[0,2].plot(x, npsin_r-interpsin_r)
axes[0,2].set_title("Numpy sin vs. table interpolation sin")

axes[1,0].plot(x, npsin_r - vdt_sin_r)
axes[1,0].set_title("Numpy sin vs. vdt sin")
axes[1,1].plot(x, npsin_r-vdt_sinv_r)
axes[1,1].set_title("Numpy sin vs. vdt vectorized sin")
axes[1,2].plot(x, npsin_r-vdt_sincos_r)
axes[1,2].set_title("Numpy sin vs. vdt sincos")

[ax.set_ylabel('Error [absolute]') for ax in axes.flatten()]
[ax.set_xlabel('x [rad]') for ax in axes.flatten()]

# plt.savefig('Errors.png')
plt.show()

In [55]:
%timeit pysin(x)
%timeit npsin(x)
%timeit cmsin(x, a)

1 loop, best of 3: 164 ms per loop
10 loops, best of 3: 27.3 ms per loop
100 loops, best of 3: 12.5 ms per loop


In [56]:
%timeit fastsincos(x, a, b)
%timeit sin_interpolated(x)

10 loops, best of 3: 26.7 ms per loop
100 loops, best of 3: 16.6 ms per loop


In [57]:
%timeit vdt_sin(x, a)
%timeit vdt_sinv(x, a)
%timeit vdt_sincos(x, a, b)

100 loops, best of 3: 5.4 ms per loop
100 loops, best of 3: 8.02 ms per loop
100 loops, best of 3: 8.13 ms per loop


***

In [36]:
%%cython --lib=gsl --lib=gslcblas

import numpy as np
cimport numpy as np
cimport cython

cdef extern from "gsl/gsl_sf_trig.h":
    cdef double gsl_sf_sin(double x)
    
def gsl_sin(double[::1] x, double[::1] s):
    
    cdef int n = x.shape[0]
    cdef int i
    for i in xrange(n):
        s[i] = gsl_sf_sin(x[i])

#     sc = sincos(x, &s[0], &c[0])

# cpdef sc(double x, np.ndarray[double, ndim=1] s, np.ndarray[double, ndim=1] c):
#     return sincos(x, &s[0], &c[0])
#     return sincos(x, <double*> s.data, <double*> c.data)

In [12]:
from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import Resonator, WakeField
from SPS import SPS


macroparticlenumber = int(10e4)
intensity = 2e11
epsn_x = 2e-6
epsn_y = 2e-6
sigma_z = 0.23


sps = SPS(machine_configuration='Q20-injection', n_segments=1, longitudinal_mode='linear',
          Qp_x=5, Qp_y=5)

bunch = sps.generate_6D_Gaussian_bunch(
    macroparticlenumber, intensity, epsn_x, epsn_y, sigma_z)

sz = bunch.sigma_z()
slices_for_wakes = UniformBinSlicer(100, z_cuts=(-3*sz, 3*sz))
reso = Resonator(7.5e6, 1.3e9, 1, 1, 0, 0, 0, switch_Z=False)
wake = WakeField(slices_for_wakes, reso)
sps.one_turn_map.append(wake)

m1, m2, m3 = sps.one_turn_map

Synchrotron init. From kwargs: n_segments = 1
Synchrotron init. From kwargs: Qp_y = 5
Synchrotron init. From kwargs: Qp_x = 5
Synchrotron init. From kwargs: longitudinal_mode = 'linear'
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


In [13]:
%timeit m1.track(bunch)
%timeit m2.track(bunch)
%timeit m3.track(bunch)

10 loops, best of 3: 18.1 ms per loop
1000 loops, best of 3: 1.82 ms per loop
100 loops, best of 3: 3.6 ms per loop


In [ ]:
# plt.ion()
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21,9))
mx = []
ex = []
for i in xrange(10000):
    for m in sps.one_turn_map:
        m.track(bunch)

    mx.append(bunch.mean_x())
    ex.append(bunch.epsn_x())
    if not i%500==0: continue
    print "\rTurn # {:d}".format(i),
    print

    # ax1.plot(bunch.x, bunch.xp, '.')
    # ax2.plot(bunch.y, bunch.yp, '.')
    # ax3.plot(bunch.z, bunch.dp, '.')
    # ax1.set_xlim(-1e-2, 1e-2)
    # ax2.set_xlim(-1e-2, 1e-2)
    # ax3.set_xlim(-1, 1)
    # ax1.set_ylim(-2e-4, 2e-4)
    # ax2.set_ylim(-2e-4, 2e-4)
    # ax3.set_ylim(-1e-2, 1e-2)

    # plt.draw()
    # plt.pause(0.1)
    # [ax.cla() for ax in [ax1, ax2, ax3]]

print
ax1.plot(mx)
ax2.plot(ex)
plt.show()
wurstel